In [9]:
%pip install pyautogen python-dotenv gradio_client pandas scikit-learn openpyxl chromadb==0.5.* sentence_transformers duckdb -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import re
import duckdb
import logging
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split

import chromadb
from chromadb.utils import embedding_functions

from autogen import AssistantAgent, UserProxyAgent, register_function

# Агент с 3 функциями:
- get_database_description
- execute_sql
- google_search

### 1. Реализация get_database_description

#### Деление на train и test примеров text2sql

In [6]:
file_path = 'few-shot_data/sql_golds.xlsx'
df = pd.read_excel(file_path)

train_df, test_df = train_test_split(df, test_size=0.5, random_state=42)
print("Кол-во данных в train", len(train_df))
print("Кол-во данных в test", len(test_df))

train_df.to_excel('few-shot_data/train_text2sql_data.xlsx', index=False)
test_df.to_excel('few-shot_data/test_text2sql_data.xlsx', index=False)

Кол-во данных в train 84
Кол-во данных в test 84


#### Создание и заполнение векторной базы данных

In [88]:
db_file = 'workdir/bank_data_user1.duckdb'
model_name = 'gpt-3.5-turbo'

CHROMADB_DIR = "vector_dbs/text2sql_few-shot"
COLLECTION_NAME = "text2sql_few-shot_examples"
chromadb_client = chromadb.PersistentClient(path=CHROMADB_DIR)

model_path = "embedding_models/e5-large"
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_path)

In [3]:
def load_data(file_path):
    return pd.read_excel(file_path)

def store_embeddings(file_path):
    data = load_data(file_path)

    collection = chromadb_client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_function
    )

    questions = data['question'].tolist()
    correct_sql = data['correct_sql'].tolist()
    ids = [str(i) for i in range(len(questions))]
    
    collection.add(
        documents=questions,
        metadatas=[{"sql": sql} for sql in correct_sql],
        ids=ids
    )

def delete_collection(collection_name):
    try:
        chromadb_client.delete_collection(name=collection_name)
        print(f"Коллекция '{collection_name}' успешно удалена.")
    except Exception as e:
        print(f"Ошибка при удалении коллекции '{collection_name}': {e}")

def get_few_shot_examples(query, top_k=5):
    collection = chromadb_client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_function
    )

    query_embedding = embedding_function([query])[0]

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    few_shot_examples = []
    for idx in range(top_k):
        few_shot_examples.append({
            'document': results['documents'][0][idx],
            'sql': results['metadatas'][0][idx]['sql']
        })
    
    return few_shot_examples

In [4]:
# For storage
file_path = "few-shot_data/train_text2sql_data.xlsx"
store_embeddings(file_path)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [4]:
query = "Как получить все заказы за последний месяц?"
top_results = get_few_shot_examples(query, top_k=3)
for pair in top_results:
    print('Вопрос:', pair['document'])
    print('SQL:', pair['sql'])

Add of existing embedding ID: 76
Add of existing embedding ID: 77
Add of existing embedding ID: 78
Add of existing embedding ID: 79
Add of existing embedding ID: 80
Add of existing embedding ID: 81
Add of existing embedding ID: 82
Add of existing embedding ID: 83
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embe

Вопрос: Сколько в среднем за месяц у меня уходит на одежду?
SQL: SELECT
  SUM(amount_value) AS monthly_spent,
  DATE_TRUNC('month', op_time) AS month
FROM
  operations_cte
WHERE
  category = 'Косметика'
  AND op_type = 'Debit'
  AND op_time >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '6 month')
  AND op_time < DATE_TRUNC('month', CURRENT_DATE)
GROUP BY
  month;
Вопрос: Сколько всего мне удалось заработать за последние три месяца
SQL: SELECT
  amount_value,
  description,
  category,
  op_time
FROM
  operations_cte
WHERE
  category = 'Зарплата'
  AND op_type = 'Credit'
  AND op_time >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '3 month');
Вопрос: Сколько в среднем я снимаю налички в месяц?
SQL: SELECT
    SUM(amount_value) AS monthly_spent,
    DATE_TRUNC('month', op_time) AS month
FROM
    operations_cte
WHERE
    category = 'Наличные'
    AND op_type = 'Debit'
    AND op_time >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '6 month')
    AND op_time < DATE_TRUNC('month', CURRENT_D

In [6]:
# delete_collection(COLLECTION_NAME)

#### Добавление примеров строчек из базы данных

In [5]:
def get_rows_with_columns(db_file: str, table_name: str, n: int = 5):
    conn = duckdb.connect(db_file)

    column_query = f"PRAGMA table_info({table_name})"
    columns = [row[1] for row in conn.execute(column_query).fetchall()]

    query = f"SELECT * FROM {table_name} LIMIT {n}"
    result = conn.execute(query).fetchall()

    column_names = " | ".join(columns)
    rows_text = "\n".join([" | ".join(map(str, row)) for row in result])

    result_text = f"Columns: {column_names}\n{rows_text}"

    conn.close()

    return result_text

In [6]:
def delete_category_collection(collection_name):
    try:
        category_chromadb_client.delete_collection(name=collection_name)
        print(f"Коллекция '{collection_name}' успешно удалена.")
    except Exception as e:
        print(f"Ошибка при удалении коллекции '{collection_name}': {e}")

In [30]:
CATEGORY_CHROMADB_DIR = "vector_dbs/data_samples"
CATEGORY_COLLECTION_NAME = "category_data_samples"

category_chromadb_client = chromadb.PersistentClient(path=CATEGORY_CHROMADB_DIR)

category_collection = chromadb_client.get_or_create_collection(
        name=CATEGORY_COLLECTION_NAME,
        embedding_function=embedding_function
    )

conn = duckdb.connect(db_file)
table_name = 'operations_cte'
unique_categories_query = f"SELECT DISTINCT category FROM {table_name}"
categories = [row[0] for row in conn.execute(unique_categories_query).fetchall() if row[0]]
conn.close()

ids = [str(i) for i in range(len(categories))]
category_collection.add(documents=categories, ids=ids)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [31]:
# Удаление всех данных из коллекции
# category_collection.delete([str(x) for x in range(category_collection.count())])

In [32]:
category_collection.count()

31

In [ ]:
categories

In [132]:
def get_similar_categories(db_file: str, user_query: str, top_n: int = 3) -> str:
    table_name = 'operations_cte'
    conn = duckdb.connect(db_file)

    user_embedding = embedding_function([user_query])[0]

    results = category_collection.query(query_embeddings=[user_embedding], n_results=top_n)
    top_categories = results['documents'][0]

    example_texts = []
    for category in top_categories:
        examples_query = f"SELECT * FROM {table_name} WHERE category = ? LIMIT 1"
        examples = conn.execute(examples_query, [category]).fetchall()

        example_rows = "\n".join([" | ".join(map(str, row)) for row in examples])
        example_texts.append(f"Category: {category}\n{example_rows}")

    result_text = "\n\n".join(example_texts)

    conn.close()

    return result_text

In [133]:
print(get_similar_categories(db_file, 'Сколько я потратила на дом и ремонт в 2024?'))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.23it/s]

Category: Дом и ремонт
RUB | -34.0 | None | *XXXX | Дом и ремонт | Grafit Bukharestskaya | 5200 | 2024-08-02 20:01:00 | Debit | OK

Category: Местный транспорт
RUB | -54.0 | None | *XXXX | Местный транспорт | Кубань Экспресс-Пригород | 4111 | 2024-12-09 22:53:16 | Debit | OK

Category: Ж/д билеты
RUB | -1681.5 | None | *XXXX | Ж/д билеты | РЖД | 4112 | 2024-12-10 07:24:37 | Debit | OK


In [193]:
CATEGORY_CHROMADB_DIR = "vector_dbs/data_samples"
DESCRIPTION_COLLECTION_NAME = "description_data_samples"

category_chromadb_client = chromadb.PersistentClient(path=CATEGORY_CHROMADB_DIR)

description_collection = chromadb_client.get_or_create_collection(
        name=DESCRIPTION_COLLECTION_NAME,
        embedding_function=embedding_function
    )

conn = duckdb.connect(db_file)
table_name = 'operations_cte'
unique_description_query = f"SELECT DISTINCT description FROM {table_name}"
descriptions = [row[0] for row in conn.execute(unique_description_query).fetchall() if row[0]]
conn.close()

ids = [str(i) for i in range(len(descriptions))]
description_collection.add(documents=descriptions, ids=ids)

Batches: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,068 - Insert of existing embedding ID: 0
2024-12-14 11:38:56,073 - Insert of existing embedding ID: 1
2024-12-14 11:38:56,073 - Insert of existing embedding ID: 1
2024-12-14 11:38:56,073 - Insert of existing embedding ID: 1
2024-12-14 11:38:56,073 - Insert of existing embedding ID: 1
2024-12-14 11:38:56,073 - Inse

In [194]:
len(descriptions)

238

In [195]:
def get_similar_descriptions(db_file: str, user_query: str, top_n: int = 3) -> str:
    table_name = 'operations_cte'
    conn = duckdb.connect(db_file)

    user_embedding = embedding_function([user_query])[0]

    results = description_collection.query(query_embeddings=[user_embedding], n_results=top_n)
    top_descriptions = results['documents'][0]

    example_texts = []
    for category in top_descriptions:
        examples_query = f"SELECT * FROM {table_name} WHERE description = ? LIMIT 1"
        examples = conn.execute(examples_query, [category]).fetchall()

        example_rows = "\n".join([" | ".join(map(str, row)) for row in examples])
        example_texts.append(f"Description: {category}\n{example_rows}")

    result_text = "\n\n".join(example_texts)

    conn.close()

    return result_text

In [197]:
print(get_similar_descriptions(db_file, 'Метро'))

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.93it/s]

Description: Московский метрополитен
RUB | -100.0 | None | *XXXX | Местный транспорт | Московский метрополитен | 4111 | 2024-07-22 19:41:27 | Debit | OK

Description: Мастер и Маргарита
RUB | -390.0 | None | *XXXX | Фастфуд | Мастер и Маргарита | 5814 | 2024-07-08 20:46:21 | Debit | OK

Description: Мария П.
RUB | -2300.0 | None | *XXXX | Переводы | Мария П. | 0 | 2024-03-17 11:22:57 | Debit | OK


In [139]:
table_name = 'operations_cte'
print(get_rows_with_columns(db_file, table_name))

Columns: amount_currency | amount_value | brand | card | category | description | mcc | op_time | op_type | status
RUB | 243.0 | None | *XXXX | Переводы | Перевод между счетами | 0 | 2024-12-10 23:36:31 | Credit | OK
RUB | -243.0 | None | *XXXX | Переводы | Перевод между счетами | 0 | 2024-12-10 23:36:30 | Debit | OK
RUB | 243.0 | None | *XXXX | Переводы | Анна К. | 0 | 2024-12-10 23:36:01 | Credit | OK
RUB | 2500.0 | None | *XXXX | Переводы | Перевод между счетами | 0 | 2024-12-10 16:12:32 | Credit | OK
RUB | -2500.0 | None | *XXXX | Переводы | Перевод между счетами | 0 | 2024-12-10 16:12:31 | Debit | OK


#### Получение схемы базы данных

In [198]:
def get_database_schema_as_text(db_path: str) -> str:
    connection = duckdb.connect(database=db_path)
    schema_text = []
    try:
        
        tables = connection.execute("SHOW TABLES").fetchall()
        for table in tables:
            table_name = table[0]
            schema_text.append(f"Таблица: {table_name}")
            
            columns = connection.execute(f"PRAGMA table_info('{table_name}')").fetchall()
            for col in columns:
                schema_text.append(f"  - {col[1]}: {col[2]}")
            schema_text.append("")
    finally:
        connection.close()
    return "\n".join(schema_text)

In [199]:
print(get_database_schema_as_text(db_file))

Таблица: accounts
  - id: VARCHAR
  - name: VARCHAR
  - type: VARCHAR
  - currency: VARCHAR
  - balance: FLOAT

Таблица: operations_cte
  - amount_currency: VARCHAR
  - amount_value: FLOAT
  - brand: VARCHAR
  - card: VARCHAR
  - category: VARCHAR
  - description: VARCHAR
  - mcc: INTEGER
  - op_time: TIMESTAMP
  - op_type: VARCHAR
  - status: VARCHAR



#### Текстовое описание таблиц и колонок

In [200]:
# text_description = '''Таблица accounts: Содержит информацию о банковских счетах пользователя, включая их идентификатор, тип, валюту и текущий баланс.
# id: Уникальный идентификатор аккаунта в текстовом формате.
# name: Название аккаунта, например, имя пользователя или название счета.
# type: Тип аккаунта, например, дебетовый, кредитный или инвестиционный.
# currency: Валюта счета, например, RUB, USD.
# balance: Текущий баланс счета в числовом формате.

# Таблица operations_cte: Содержит подробные записи о транзакциях, включая сумму, категорию, описание и статус операции.
# amount_currency: Валюта операции, например, RUB или другая локальная валюта.
# amount_value: Сумма операции в числовом формате.
# brand: Название торговой марки или организации, связанной с операцией, например, 'Acer', 'Магнит'.
# card: Последние четыре цифры карты, используемой для операции, например '*0117'.
# category: Категория расхода, например, 'Фастфуд', 'Супермаркеты' или 'Переводы'.
# description: Описание операции, например, название магазина или услуги.
# mcc: Код категории продавца (MCC), указывающий на тип торговой точки.
# op_time: Время проведения операции в формате временной метки.
# op_type: Тип операции, 'Debit' (списание) или 'Credit' (зачисление).
# status: Статус операции,'OK' или 'FAILED'.
# '''

text_description = '''Таблица operations_cte: Содержит подробные записи о транзакциях, включая сумму, категорию, описание и статус операции.
amount_currency: Валюта операции, например, RUB или другая локальная валюта.
amount_value: Сумма операции в числовом формате.
brand: Название торговой марки или организации, связанной с операцией, например, 'Acer', 'Магнит'.
card: Последние четыре цифры карты, используемой для операции, например '*0117'.
category: Категория расхода, например, 'Фастфуд', 'Супермаркеты' или 'Переводы'.
description: Описание операции, например, название магазина или услуги.
mcc: Код категории продавца (MCC), указывающий на тип торговой точки.
op_time: Время проведения операции в формате временной метки.
op_type: Тип операции, 'Debit' (списание) или 'Credit' (зачисление).
status: Статус операции,'OK' или 'FAILED'.
'''

#### Объединение всей информации о базе данных

In [201]:
def get_database_description(query: str) -> str:
    # Схема
    schema_text = get_database_schema_as_text(db_file)

    # Few-shot
    few_shot_examples = 'Вот тебе примеры:\n'
    few_shot_text2sql_examples = get_few_shot_examples(query, top_k=3)
    for text2sql_example in few_shot_text2sql_examples:
        few_shot_examples += f'Вопрос: {text2sql_example["document"]}\nSQL: {text2sql_example["sql"]}\n'
    few_shot_examples += '\n'

    # Data samples
    table_name = 'operations_cte'
    data_samples = get_rows_with_columns(db_file, table_name, n= 3)

    categories_data_samples = get_similar_categories(db_file, query)
    descriptions_data_samples = get_similar_descriptions(db_file, query)

    database_description = f'''Схема базы данных в PostgreSQL: 
{schema_text}
Описание таблиц и колонок:
{text_description}

Вот тебе пример данных из таблицы: {table_name}
{data_samples}

{categories_data_samples}
{descriptions_data_samples}

Примеры вопросов к базе данных и ответов в виде SQL-запросов:
{few_shot_examples}
'''
    return database_description

In [202]:
print(get_database_description('Сколько я потратила на отели в 2024?'))

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.68it/s]


Схема базы данных в PostgreSQL: 
Таблица: accounts
  - id: VARCHAR
  - name: VARCHAR
  - type: VARCHAR
  - currency: VARCHAR
  - balance: FLOAT

Таблица: operations_cte
  - amount_currency: VARCHAR
  - amount_value: FLOAT
  - brand: VARCHAR
  - card: VARCHAR
  - category: VARCHAR
  - description: VARCHAR
  - mcc: INTEGER
  - op_time: TIMESTAMP
  - op_type: VARCHAR
  - status: VARCHAR

Описание таблиц и колонок:
Таблица operations_cte: Содержит подробные записи о транзакциях, включая сумму, категорию, описание и статус операции.
amount_currency: Валюта операции, например, RUB или другая локальная валюта.
amount_value: Сумма операции в числовом формате.
brand: Название торговой марки или организации, связанной с операцией, например, 'Acer', 'Магнит'.
card: Последние четыре цифры карты, используемой для операции, например '*0117'.
category: Категория расхода, например, 'Фастфуд', 'Супермаркеты' или 'Переводы'.
description: Описание операции, например, название магазина или услуги.
mcc: Ко

### 2. Реализация execute_sql

In [203]:
def execute_query(sql_query: str, table_name: str) -> str:
    db_path = db_file
    # Удаляем ; в конце запроса, если она есть
    if sql_query.strip().endswith(';'):
        sql_query = sql_query.strip()[:-1]

    try:
        conn = duckdb.connect(db_path)
        # Выполняем запрос и создаём таблицу
        conn.execute(f"CREATE TABLE {table_name} AS ({sql_query})")
        
        # Извлекаем имена колонок и 3 строки из таблицы
        columns_query = f"PRAGMA table_info({table_name})"
        columns = [row[1] for row in conn.execute(columns_query).fetchall()]
        
        examples_query = f"SELECT * FROM {table_name} LIMIT 3"
        examples = conn.execute(examples_query).fetchall()
        
        conn.close()
        
        # Формируем текст с примерами
        examples_text = "\nПримеры 3 строк (" + ", ".join(columns) + "):\n"
        for row in examples:
            examples_text += "  " + str(row) + "\n"

        return f"Данные успешно сохранены в таблицу: {table_name}{examples_text}"
    except Exception as e:
        return f"Ошибка при выполнении запроса: {e}"


In [204]:
def delete_table(database_path: str, table_name: str):
    try:
        conn = duckdb.connect(database_path)

        conn.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Таблица '{table_name}' успешно удалена (если существовала).")

    except Exception as e:
        print(f"Ошибка при удалении таблицы: {e}")

    finally:
        conn.close()

In [205]:
table_name = 'operations_cte'
table_name_by_sql = 'selected_10_rows'
output_csv_path = 'test.csv'
print(execute_query(f"SELECT * FROM {table_name} LIMIT 10", table_name_by_sql))
delete_table(db_file, table_name_by_sql)

Данные успешно сохранены в таблицу: selected_10_rows
Примеры 3 строк (amount_currency, amount_value, brand, card, category, description, mcc, op_time, op_type, status):
  ('RUB', 243.0, None, '*XXXX', 'Переводы', 'Перевод между счетами', 0, datetime.datetime(2024, 12, 10, 23, 36, 31), 'Credit', 'OK')
  ('RUB', -243.0, None, '*XXXX', 'Переводы', 'Перевод между счетами', 0, datetime.datetime(2024, 12, 10, 23, 36, 30), 'Debit', 'OK')
  ('RUB', 243.0, None, '*XXXX', 'Переводы', 'Анна К.', 0, datetime.datetime(2024, 12, 10, 23, 36, 1), 'Credit', 'OK')

Таблица 'selected_10_rows' успешно удалена (если существовала).


#### 3. Реализация fetch_table_data

In [206]:
def fetch_table_data(table_name: str) -> str:
    database_path = db_file
    try:
        # Connect to the DuckDB database
        conn = duckdb.connect(database_path)

        # Verify if the table exists
        tables = conn.execute("SHOW TABLES").fetchall()
        table_names = [row[0] for row in tables]
        if table_name not in table_names:
            raise ValueError(f"Table '{table_name}' does not exist in the database.")

        # Fetch data from the specified table
        query = f"SELECT * FROM {table_name} LIMIT 10"
        result = conn.execute(query).fetchall()
        columns = [desc[0] for desc in conn.description]

        # Format the result as a text string
        text_result = "\t".join(columns) + "\n"  # Add column headers
        for row in result:
            text_result += "\t".join(map(str, row)) + "\n"

        return text_result
    except Exception as e:
        raise e
    finally:
        conn.close()

In [207]:
print(fetch_table_data('operations_cte'))

amount_currency	amount_value	brand	card	category	description	mcc	op_time	op_type	status
RUB	243.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 23:36:31	Credit	OK
RUB	-243.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 23:36:30	Debit	OK
RUB	243.0	None	*XXXX	Переводы	Анна К.	0	2024-12-10 23:36:01	Credit	OK
RUB	2500.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 16:12:32	Credit	OK
RUB	-2500.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 16:12:31	Debit	OK
RUB	2500.0	None	*XXXX	Переводы	Илья К.	0	2024-12-10 14:55:44	Credit	OK
RUB	4139.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 14:10:49	Credit	OK
RUB	-4139.0	None	*XXXX	Переводы	Перевод между счетами	0	2024-12-10 14:10:48	Debit	OK
RUB	4000.0	None	*XXXX	Переводы	Анна К.	0	2024-12-10 14:09:59	Credit	OK
RUB	-1681.5	None	*XXXX	Ж/д билеты	РЖД	4112	2024-12-10 07:24:37	Debit	OK



### Реализация агента

In [208]:
def is_end_conversation(msg: str) -> bool:
    if msg.get("content"):
        if "'Ответ:'" in msg["content"]:
            return False
        if 'Ответ:' in msg["content"]:
            return True
        if 'TERMINATE' in msg["content"]:
            return True
    return False

In [209]:
def get_current_date():
    current_date = datetime.now()
    return current_date.strftime("%d день, %m месяц, %Y год")

In [210]:
word_info = f'''Дополнительная инфрмация: текущая дата {get_current_date()}'''

In [211]:
question_answer_examples = '''Пример:
Вопрос: Сколько я потратил на отели в 2024?
Ответ: На отели вами было потрачено 9420 рублей.
Вопрос: Когда был последний перевод?
Ответ: Последний перевод был совершен на сумму 243 RUB и произошел 10 декабря 2024 года в 23:36.
Вопрос: Сколько я потратила в Пятёрочке?
Ответ: Ты потратила 12934.98 RUB в Пятёрочке'''

additional_instructions = '''Если речь идёт о расходах, то не используй минус при ответе.
Если из базы данных возвращается данных, то отвечай, что не было найдено подходящих данных.'''

llm_config = {"config_list": [{"model": model_name, "api_key": os.environ["OPENAI_API_KEY"]}]}

assistant = AssistantAgent("assistant", llm_config=llm_config, 
                           system_message='''Ты профессиональный финансовый ассистент. Ты создан отвечать на вопросы пользователей об их финансах, быть полезным.
Действуй по следующему плану:
1. Посмотри информацию о базе данных
2. Сделай промежуточные таблицы
3. Сделай выбор нужных таблиц для ответа на вопрос и сагрегируй результат

{additional_instructions}

{question_answer_examples}

Ответ давай подробно, указывая все необходимые данные.
Если в ответе есть число денег, то укажи валюту.
Если в ответе есть различные операции, то укажи при наличии информацию когда и в каком магазине они были совершены.
Если чего-то не знаешь, то не пиши этого.
{word_info}''')

user_proxy = UserProxyAgent("user_proxy",
                            human_input_mode="NEVER",
                            code_execution_config={"work_dir": "workdir", "use_docker": False},
                            is_termination_msg=is_end_conversation
                            )

In [212]:
register_function(
    get_database_description,
    caller=assistant,
    executor=user_proxy,
    name="get_database_description",
    description="Получение информации о базе данных. В качестве параметров передаётся текстовый запрос с указанием вопроса к данным. Пример: 'Последние транзакции', 'Покупки в магазине Магнит', 'Траты в Вкусно и Точка'"
)

In [213]:
register_function(
    execute_query,
    caller=assistant,
    executor=user_proxy,
    name="execute_query",
    description="Исполнение SQL запроса к базе данных PostgreSQL. При составлении SQL выбирай как можно больше столбцов, чтобы дать качественее ответ. В качестве параметров передаётся SQL-запрос и таблица, в которую сохраняется результат (используй для промежуточных вычислений)."
)

In [214]:
register_function(
    fetch_table_data,
    caller=assistant,
    executor=user_proxy,
    name="fetch_table_data",
    description="Извлечение 10 строк данных из определённой таблицы. Название таблицы указывается как аргумент."
)

In [215]:
def list_and_clean_tables(database_path: str, base_tables: list):
    try:
        connection = duckdb.connect(database_path)
        
        tables = connection.execute("SHOW TABLES").fetchall()

        print("Список таблиц в базе данных:")
        for table in tables:
            print(table[0])
        
        for table in tables:
            if table[0] not in base_tables:
                connection.execute(f"DROP TABLE IF EXISTS {table[0]}")
                print(f"Таблица '{table[0]}' удалена.")
                
    except Exception as e:
        print(f"Ошибка: {e}")
    
    finally:
        connection.close()

In [216]:
BASE_TABLES = ['accounts', 'operations_cte']

log_dir = "chat_logs"
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log')
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    encoding='utf-8'
)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
logging.getLogger().addHandler(console_handler)

def get_agent_answer(question: str) -> str:
    logging.info('question: %s', question)

    try:
        chatresult = user_proxy.initiate_chat(assistant,
                                                message=f'''Тебе дан следующий диалог с пользователем: {question}
Для ответа используй извлечение данных из какой-либо таблицы из базы данных.
В ответе упомяни промежуточные вычисления, если такие были.
Формат ответа: 'Ответ:'
В конце ответа пиши TERMINATE.''')
        agent_answer = chatresult.summary.split('Ответ:')[-1].strip()
        logging.info('agent_answer: %s', agent_answer)
        list_and_clean_tables(db_file, BASE_TABLES)
        return agent_answer.rstrip('.')
    except Exception as e:
        logging.error('Ошибка: %s', str(e))
        return 'Извините, при вашем обращении произошла ошибка. Уже чиним это 😊'


In [223]:
question = 'Сколько я потратила в магазине Планета Здоровья?'
agent_answer = get_agent_answer(question)

2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазине Планета Здоровья?
2024-12-14 11:43:23,459 - question: Сколько я потратила в магазин

user_proxy (to assistant):

Тебе дан следующий диалог с пользователем: Сколько я потратила в магазине Планета Здоровья?
Для ответа используй извлечение данных из какой-либо таблицы из базы данных.
В ответе упомяни промежуточные вычисления, если такие были.
Формат ответа: 'Ответ:'
В конце ответа пиши TERMINATE.

--------------------------------------------------------------------------------


2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:25,302 - HTTP Request: POST h

assistant (to user_proxy):

***** Suggested tool call (call_fsV5Z1zxbzfMIH28JDCinRis): get_database_description *****
Arguments: 
{"query":"Потраченные деньги в магазине Планета Здоровья"}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_database_description...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.88it/s]

...

assistant (to user_proxy):

***** Suggested tool call (call_AwGxYqbD9qnDGH9opwJwb9is): fetch_table_data *****
Arguments: 
{"table_name":"total_spent_planeta_zdorovya"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_table_data...
user_proxy (to assistant):

***** Response from calling tool (call_AwGxYqbD9qnDGH9opwJwb9is) *****
total_spent
-2075.0

**********************************************************************

--------------------------------------------------------------------------------


2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-14 11:43:29,851 - HTTP Request: POST h

assistant (to user_proxy):

Ответ: 
Вы потратили -2075.0 RUB в магазине Планета Здоровья.
TERMINATE.

--------------------------------------------------------------------------------


2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: Вы потратили -2075.0 RUB в магазине Планета Здоровья.
.
2024-12-14 11:43:30,030 - agent_answer: 

Список таблиц в базе данных:
accounts
operations_cte
total_spent_planeta_zdorovya
Таблица 'total_spent_planeta_zdorovya' удалена.


In [224]:
print(agent_answer)

Вы потратили -2075.0 RUB в магазине Планета Здоровья.



In [179]:
chatresult.cost['usage_including_cached_inference']['total_cost']

0.020204999999999997

In [96]:
list_and_clean_tables(db_file, BASE_TABLES)

Список таблиц в базе данных:
accounts
operations_cte


In [158]:
db_path = db_file

conn = duckdb.connect(db_path)
result_df = conn.execute(f"SELECT amount_value, description, category, op_time FROM operations_cte WHERE category = 'Авиабилеты' AND op_type = 'Debit' ORDER BY op_time DESC LIMIT 1").fetch_df()
conn.close()
result_df   

,amount_value,description,category,op_time


In [113]:
conn = duckdb.connect(db_file)
result_df = conn.execute(f"SELECT * FROM operations_cte WHERE category='Фастфуд'").fetch_df()
conn.close()
result_df

,amount_currency,amount_value,brand,card,category,description,mcc,op_time,op_type,status
0,RUB,-15.0,None,*XXXX,Фастфуд,Кафе ООО Премьер-Ивент г. Сочи,5814,2024-12-09 09:53:25,Debit,OK
1,RUB,-180.0,None,*XXXX,Фастфуд,Вкусно — и точка,5814,2024-12-08 13:19:54,Debit,OK
2,RUB,-230.0,None,*XXXX,Фастфуд,Кафе ООО Премьер-Ивент г. Сочи,5814,2024-10-17 11:54:03,Debit,OK
3,RUB,-260.0,None,*XXXX,Фастфуд,Кафе ООО Премьер-Ивент г. Сочи,5814,2024-09-10 12:50:38,Debit,OK
4,RUB,-120.0,None,*XXXX,Фастфуд,Университет Сириус,5814,2024-09-09 18:23:25,Debit,OK
...,...,...,...,...,...,...,...,...,...,...
93,RUB,-290.0,None,*XXXX,Фастфуд,Sushi Boofet,5814,2024-01-30 13:48:21,Debit,OK
94,RUB,-220.0,None,*XXXX,Фастфуд,Etlon Coffee,5814,2024-01-27 20:29:28,Debit,OK
95,RUB,-123.0,None,*XXXX,Фастфуд,Теремок,5814,2024-01-23 10:35:29,Debit,OK
96,RUB,-38.0,None,*XXXX,Фастфуд,Первая полоса,5814,2024-01-22 19:52:00,Debit,OK


In [59]:
result_df

,amount_currency,amount_value,brand,card,category,description,mcc,op_time,op_type,status
0,RUB,-3578.0,None,*XXXX,Авиабилеты,Аэрофлот,3011,2024-11-21 07:15:14,Credit,OK
1,RUB,6273.0,None,*XXXX,Авиабилеты,Ural Airlines,4511,2024-11-19 16:07:49,Credit,OK
2,RUB,-6673.0,None,*XXXX,Авиабилеты,Ural Airlines,4511,2024-10-30 18:47:08,Credit,OK
3,RUB,-1000.0,None,*XXXX,Авиабилеты,Ural Airlines,4511,2024-09-08 11:14:50,Credit,OK
4,RUB,-10782.0,None,*XXXX,Авиабилеты,Ural Airlines,4511,2024-08-08 20:45:22,Credit,OK


In [47]:
BASE_TABLES = ['accounts', 'operations_cte']

connection = duckdb.connect(db_file)

# Запрос для получения всех таблиц
tables = connection.execute("SHOW TABLES").fetchall()

# Вывод списка таблиц
print("Список таблиц в базе данных:")
for table in tables:
    print(table[0])

for table in tables:
    if table not in BASE_TABLES:
        delete_table(db_file, table)

Список таблиц в базе данных:
accounts
operations_cte
Ошибка при удалении таблицы: Parser Error: syntax error at or near "("
Ошибка при удалении таблицы: Parser Error: syntax error at or near "("


In [20]:
for table in tables:
    if table not in BASE_TABLES:
        delete_table('workdir/finance_data.duckdb', table)

Таблица 'last_flight_purchase' успешно удалена (если существовала).
